In [0]:
import dlt
import requests
from pyspark.sql.functions import col, round, when, lit
from pyspark.sql.types import DoubleType, StringType, StructType, StructField
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

def get_exchange_rate(currency_code):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency_code}-BRL"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pair_key = currency_code + 'BRL'
        return float(data[pair_key]['bid'])
    return 1.0

@dlt.table(
    name="prices",
    comment="dados da bigtables gerando uma tabela com dados de preço"
)
def playtime_silver():
    df = dlt.read("steamdatabricks_workspace.gold.bigtable")
    df_prices = df.select('appid', 'Currency', 'Price', 'is_free').distinct()

    # Obter moedas únicas (excluindo '-')
    unique_currencies = [row.Currency for row in df_prices.select("Currency").distinct().collect() if row.Currency != '-']

    # Criar lista de tuplas (currency, rate)
    rates_list = [(currency, get_exchange_rate(currency)) for currency in unique_currencies]
    rates_list.append(('-', 1.0))  # tratar '-' como BRL

    # Criar schema para DataFrame de taxas
    schema = StructType([
        StructField('Currency', StringType(), False),
        StructField('Rate', DoubleType(), False)
    ])

    # Criar DataFrame de taxas
    df_rates = spark.createDataFrame(rates_list, schema)

    # Fazer join para associar taxas aos preços
    df_joined = df_prices.join(df_rates, on='Currency', how='left')

    # Converter o preço para BRL usando colunas e expressão Spark
    df_converted = df_joined.withColumn(
        'PriceBRL',
        when(col('is_free') == True, 0.0)
        .otherwise(round(col('Price') * col('Rate'), 2))
    )

    # Atualizar coluna is_free caso price seja zero
    df_final = df_converted.withColumn(
        'is_free',
        when(col('Price') == 0, True).otherwise(col('is_free'))
    )

    return df_final
